In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
from pprint import pprint
import json
from scipy.stats import mode
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

%matplotlib inline

In [2]:
result_files = glob.glob('../results/baseline-daic-woz-modalities+class-weights/*.csv')

dfs = []
for file in result_files:
    df = pd.read_csv(file)
    df['filename'] = file.split('/')[-1]

    dfs.append(df)

dfs = pd.concat(dfs)


In [3]:
dfs["prediction_kind"].unique()

array(['last', 'mean', 'mode', 'threshold', 'mode_threshold',
       'last_presence', 'mean_presence', 'mode_presence',
       'threshold_presence', 'mode_threshold_presence'], dtype=object)

In [19]:
evals = {}
for modality_id in ['audio', 'video', 'audiovisual']:
    for eval_id in ['last', 'mean', 'mode', 'threshold', 'mode_threshold']:
        evals[f'{modality_id}:{eval_id}'] = dfs[dfs['prediction_kind'] == eval_id]
        evals[f'{modality_id}:{eval_id}'] = evals[f'{modality_id}:{eval_id}'][evals[f'{modality_id}:{eval_id}']['filename'].str.contains(modality_id+'-')]
        evals[f'{modality_id}:{eval_id}'] = evals[f'{modality_id}:{eval_id}'][evals[f'{modality_id}:{eval_id}']['dataset_kind'] == 'test']
        evals[f'{modality_id}:{eval_id}'].sort_values(by=['seconds_per_window', 'presence_threshold', 'run_id'], inplace=True)

In [30]:
evals_audio = evals['audio:mode_threshold']
evals_video = evals['video:mode_threshold']
evals_audiovisual = evals['audiovisual:mode_threshold']


In [45]:
evals_video.groupby(['seconds_per_window', 'presence_threshold', 'model_args.num_layers', 'model_args.self_attn_num_heads', 'model_args.self_attn_dim_head']).agg(
    {'run_id': 'count', 'f1': ['mean', 'std'], 'f1_weighted': ['mean', 'std'], 'precision': ['mean', 'std'], 'recall': ['mean', 'std'], 'accuracy': ['mean', 'std']}).reset_index()

seconds_per_window presence_threshold model_args.num_layers  \
                                                                
0                  7               0.25                     8   

  model_args.self_attn_num_heads model_args.self_attn_dim_head run_id  \
                                                                count   
0                              8                            32      5   

         f1           f1_weighted          precision           recall  \
       mean       std        mean      std      mean       std   mean   
0  0.428723  0.073746     0.61126  0.08547  0.388914  0.080215    0.5   

             accuracy            
        std      mean       std  
0  0.133631  0.604255  0.091018